In [3]:
from dotenv import load_dotenv
load_dotenv()

import operator

In [4]:
import operator
import logging
import os
from typing import Annotated, Literal, TypedDict, Sequence
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, FunctionMessage
from langchain_core.tools import BaseTool, Tool
from langchain_core.runnables import Runnable, RunnableConfig
from langchain_core.language_models import LanguageModelLike
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolExecutor
from langgraph.checkpoint import MemorySaver
from utils.prompt_loader import load_markdown_prompt
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain_community.tools.wolfram_alpha import WolframAlphaQueryRun
from langgraph.prebuilt.tool_executor import ToolInvocation
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from utils.llm_setup import get_llm

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Define the state
class MetaPromptingState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]
    error_log: list[str]
    turn_count: int

# Load meta-prompter instructions
META_PROMPTER_INSTRUCTIONS = load_markdown_prompt("../prompts/meta-prompter.md")

MAX_TURNS = 15  # Increased maximum number of turns for more complex queries

def create_meta_prompting_agent(
    model: LanguageModelLike,
    tools: Sequence[BaseTool],
    checkpointer: MemorySaver = None,
):
    # Create ToolExecutor
    tool_executor = ToolExecutor(tools)

    # Create the meta-prompter node
    def meta_prompter(state: MetaPromptingState, config: RunnableConfig):
        messages = state['messages']
        turn_count = state.get('turn_count', 0) + 1
        
        if turn_count > MAX_TURNS:
            logger.info(f"Reached maximum turns ({MAX_TURNS}). Forcing end of conversation.")
            return {
                "messages": [AIMessage(content="I apologize, but I've been unable to provide a satisfactory answer within a reasonable number of steps. Here's my best attempt at a final answer based on what we've discussed: [Summary of the conversation]")],
                "turn_count": turn_count,
                "error_log": state.get("error_log", []) + ["Reached maximum turns"],
            }
        
        response = model.invoke(
            [
                HumanMessage(content=f"{META_PROMPTER_INSTRUCTIONS}\n\nRemember to use available tools for up-to-date information when necessary. When you have a final answer, start your response with 'FINAL ANSWER:' and be sure it's comprehensive."),
                *messages
            ],
            config
        )
        
        result = {
            "messages": [AIMessage(content=f"Meta-Prompter: {response.content}")],
            "turn_count": turn_count,
            "error_log": state.get("error_log", []),
        }
        
        return result

    # Create the expert node with ReAct-like behavior
    def expert_node(state: MetaPromptingState, config: RunnableConfig):
        messages = state['messages']
        tool_names = ", ".join([tool.name for tool in tools])
        
        expert_prompt = f"""You are an expert assistant with access to the following tools: {tool_names}. 
        Use them when necessary to provide accurate and up-to-date information. 
        To use a tool, respond with the tool name and input in the following format:
        Tool: <tool_name>
        Input: <tool_input>
        
        If you have a final answer, start your response with 'FINAL ANSWER:' and ensure it's comprehensive.
        
        Current conversation:
        {messages}
        
        What would you like to do next? Consider using a tool if you need current information or specific data."""
        
        response = model.invoke([HumanMessage(content=expert_prompt)], config)
        
        if "Tool:" in response.content and "Input:" in response.content:
            tool_name = response.content.split("Tool:")[1].split("\n")[0].strip()
            tool_input = response.content.split("Input:")[1].strip()
            
            logger.info(f"Using tool: {tool_name}")
            
            try:
                tool_invocation = ToolInvocation(tool=tool_name, tool_input=tool_input)
                tool_result = tool_executor.invoke(tool_invocation)
                
                result = {
                    "messages": [
                        AIMessage(content=response.content),
                        FunctionMessage(content=str(tool_result), name=tool_name)
                    ],
                    "error_log": state.get("error_log", [])
                }
            except Exception as e:
                error_message = f"Error executing {tool_name}: {str(e)}"
                logger.error(error_message)
                result = {
                    "messages": [AIMessage(content=f"I encountered an error while trying to use the {tool_name} tool. I'll try a different approach.")],
                    "error_log": state.get("error_log", []) + [error_message]
                }
        else:
            result = {
                "messages": [AIMessage(content=response.content)],
                "error_log": state.get("error_log", [])
            }
        
        result["turn_count"] = state.get("turn_count", 0)
        return result

    # Define the function to determine whether to continue or end
    def should_continue(state: MetaPromptingState) -> Literal["continue", "expert", "end"]:
        last_message = state['messages'][-1].content if state['messages'] else ""
        
        if "FINAL ANSWER:" in last_message:
            logger.info("FINAL ANSWER detected, ending conversation")
            return "end"
        elif state.get("turn_count", 0) >= MAX_TURNS:
            logger.info("Maximum turns reached, ending conversation")
            return "end"
        elif last_message.startswith("Meta-Prompter:"):
            return "expert"
        else:
            return "continue"

    # Create the graph
    workflow = StateGraph(MetaPromptingState)

    # Add nodes
    workflow.add_node("meta_prompter", meta_prompter)
    workflow.add_node("expert", expert_node)

    # Set entry point
    workflow.set_entry_point("meta_prompter")

    # Add edges
    workflow.add_conditional_edges(
        "meta_prompter",
        should_continue,
        {
            "continue": "meta_prompter",
            "expert": "expert",
            "end": END
        }
    )
    workflow.add_conditional_edges(
        "expert",
        should_continue,
        {
            "continue": "meta_prompter",
            "expert": "expert",
            "end": END
        }
    )

    # Compile the graph
    return workflow.compile(checkpointer=checkpointer)

# Example usage
if __name__ == "__main__":
    # Create the language model

    model = get_llm(provider="openai", model_name="gpt-4o")
    # model = get_llm(provider="claude", model_name="claude-3-5-sonnet-20240620")

    # Create instances of the tools
    tools = [
        TavilySearchResults(max_results=3),
    ]

    # Try to add Wolfram Alpha tool if available
    wolfram_alpha_appid = os.getenv("WOLFRAM_ALPHA_APPID")
    
    if wolfram_alpha_appid:
        try:
            wolfram = WolframAlphaAPIWrapper()
            tools.append(WolframAlphaQueryRun(api_wrapper=wolfram))
            logger.info("Wolfram Alpha tool added successfully.")
        except Exception as e:
            logger.warning(f"Failed to initialize Wolfram Alpha tool: {str(e)}")
    else:
        logger.warning("WOLFRAM_ALPHA_APPID not set. Skipping Wolfram Alpha tool.")

    # Try to add Wikipedia tool if available
    try:
        wikipedia = WikipediaAPIWrapper()
        tools.append(Tool(
            name="Wikipedia",
            func=wikipedia.run,
            description="Useful for querying Wikipedia to get information on a wide range of topics."
        ))
        logger.info("Wikipedia tool added successfully.")
    except ImportError:
        logger.warning("Wikipedia package not found. Proceeding without Wikipedia tool.")
    
    agent = create_meta_prompting_agent(model, tools)
    
    # Use streaming with improved output formatting
    try:
        complex_query = """
        Compare the economic impact of renewable energy adoption in Germany and China over the last decade. 
        Include data on their current energy mix, major renewable projects, and how this shift has affected 
        their carbon emissions and job markets. Also, provide a brief forecast of their renewable energy goals 
        for the next 5 years.
        """
        for chunk in agent.stream({"messages": [HumanMessage(content=complex_query)], "error_log": [], "turn_count": 0}):
            for key, value in chunk.items():
                if 'messages' in value:
                    for message in value['messages']:
                        print(f"\n{'=' * 40}")
                        print(f"{message.type.capitalize()}:")
                        print(f"{'=' * 40}")
                        print(message.content)
                elif key == 'error_log':
                    print(f"\n{'=' * 40}")
                    print("Error Log:")
                    print(f"{'=' * 40}")
                    for error in value:
                        print(error)
                else:
                    print(f"\n{'=' * 40}")
                    print(f"{key.capitalize()}:")
                    print(f"{'=' * 40}")
                    for sub_key, sub_value in value.items():
                        print(f"{sub_key}: {sub_value}")
            print("\n" + "-" * 80 + "\n")
    except Exception as e:
        logger.exception("An error occurred during execution")
        raise

2024-07-16 23:17:35,494 - INFO - Wolfram Alpha tool added successfully.
2024-07-16 23:17:35,495 - INFO - Wikipedia tool added successfully.
2024-07-16 23:17:40,907 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Ai:
Meta-Prompter: To address this complex question, I will need to consult multiple experts: Expert Researcher, Expert Statistician, and Expert Economist. Here is how I plan to break down the task:

1. **Expert Researcher**: Gather data on the current energy mix, major renewable projects, carbon emissions, and job market impacts for both Germany and China over the last decade.
2. **Expert Statistician**: Analyze the gathered data to identify trends and changes over time.
3. **Expert Economist**: Evaluate the economic impact of renewable energy adoption and provide a forecast for the next 5 years.

Let's start with the data gathering.

### Step 1: Data Collection
I will first consult the Expert Researcher.

```
Expert Researcher:
"""
Please gather the following data for both Germany and China:

1. Current energy mix, including percentages of renewable and non-renewable sources.
2. Major renewable energy projects initiated or completed in the last decade.
3. Changes in carbon emissions

2024-07-16 23:17:45,099 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-16 23:17:45,106 - INFO - Using tool: Wikipedia



Ai:
To gather the necessary data, I will use a combination of tools. Let's start with collecting information about the current energy mix, major renewable energy projects, changes in carbon emissions, and job market impacts for both Germany and China.

### Step 1: Gather Data for Germany
1. Current energy mix
2. Major renewable energy projects
3. Changes in carbon emissions
4. Impact on job markets
5. Renewable energy goals for the next 5 years

### Step 2: Gather Data for China
1. Current energy mix
2. Major renewable energy projects
3. Changes in carbon emissions
4. Impact on job markets
5. Renewable energy goals for the next 5 years

I will start by gathering the current energy mix and major renewable energy projects for Germany and China.

#### Data Collection for Germany

Tool: Wikipedia
Input: "Energy in Germany"

Tool: Wikipedia
Input: "Renewable energy in Germany"

#### Data Collection for China

Tool: Wikipedia
Input: "Energy in China"

Tool: Wikipedia
Input: "Renewable energ

2024-07-16 23:18:03,121 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...


Ai:
Meta-Prompter: Meta-Prompter: Given the repeated lack of results from automated searches, I will proceed by directly consulting the Expert Researcher for comprehensive information.

```
Expert Researcher:
"""
Please provide the following comprehensive data for both Germany and China:

1. Current energy mix (2023), including percentages of renewable and non-renewable sources.
2. Major renewable energy projects initiated or completed in the last decade.
3. Changes in carbon emissions over the last decade.
4. Impact on job markets due to the shift towards renewable energy.
5. Renewable energy goals for the next 5 years.

For each point, provide separate data for Germany and China.
"""
```

I will wait for the Expert Researcher to provide the necessary information.

--------------------------------------------------------------------------------



2024-07-16 23:19:17,911 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-16 23:19:17,915 - INFO - Using tool: Wolfram Alpha



Ai:
Given the challenges in obtaining detailed data from the tools used, I recommend using a different tool to gather the necessary information. Let's use Wolfram Alpha, which often provides comprehensive and up-to-date data on various topics, including energy statistics.

### Step 1: Gather Comprehensive Data for Germany

1. **Current energy mix (2023)**
2. **Major renewable energy projects initiated or completed in the last decade**
3. **Changes in carbon emissions over the last decade**
4. **Impact on job markets due to the shift towards renewable energy**
5. **Renewable energy goals for the next 5 years**

### Step 2: Gather Comprehensive Data for China

1. **Current energy mix (2023)**
2. **Major renewable energy projects initiated or completed in the last decade**
3. **Changes in carbon emissions over the last decade**
4. **Impact on job markets due to the shift towards renewable energy**
5. **Renewable energy goals for the next 5 years**

Let's start by querying Wolfram Alpha f

2024-07-16 23:19:19,337 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"
2024-07-16 23:19:19,351 - ERROR - An error occurred during execution
Traceback (most recent call last):
  File "/var/folders/c_/vbgn7b4d1pq8rdj99dvcthbm0000gn/T/ipykernel_5053/3215920526.py", line 221, in <module>
    for chunk in agent.stream({"messages": [HumanMessage(content=complex_query)], "error_log": [], "turn_count": 0}):
  File "/Users/edbalogh/Projects/automap_ai/.pyenv/lib/python3.11/site-packages/langgraph/pregel/__init__.py", line 1110, in stream
    _panic_or_proceed(done, inflight, step)
  File "/Users/edbalogh/Projects/automap_ai/.pyenv/lib/python3.11/site-packages/langgraph/pregel/__init__.py", line 1780, in _panic_or_proceed
    raise exc
  File "/Users/edbalogh/anaconda3/lib/python3.11/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/edbalo

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'messages[28].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[28].name', 'code': 'invalid_value'}}